<font size=5>Run Text_Matcher Algorithm</font>

In [ ]:
#Troubleshooting: !jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 if you 
# get a message about the data rate limit


# import libraries needed
import sys

import pandas as pd
import numpy as np


# TBD: Some of these libraries are relics of earlier notebooks.
# Reactivate any of the following that are needed, delete any that aren't needed.

try:
   import nltk
except:   
   !{sys.executable} -m pip install nltk

try:
   import termcolor
except:   
   !{sys.executable} -m pip install termcolor

from matcher import Text, Matcher


try:   
   from IPython.display import clear_output
except:
   !{sys.executable} -m pip install IPython.display 
   from IPython.display import clear_output

try:
   from matcher import Text, Matcher
except:
   !{sys.executable} -m pip install matcher
   from matcher import Text, Matcher

try:
    import re
except:
    !{sys.executable} -m pip install re
    import re


try:
    import json
except:
    !{sys.executable} -m pip install json
    import json

try:
    import ipywidgets as widgets
except:
    !{sys.executable} -m pip install ipywidgets
    import ipywidgets as widgets

#%pip install altair

try:
    import altair as alt
except:
    !{sys.executable} -m pip install altair
    import altair as alt


try:
    from pathlib import Path
except:
    !{sys.executable} -m pip install pathlib
    from pathlib import Path



# from IPython.display import display

try:
    from IPython.display import display
except:
    !{sys.executable} -m pip install IPython.display
    from IPython.display import display

#new viz library for single-column heatmap    

try:
    import matplotlib.pyplot as plt
except:
    !{sys.executable} -m pip install matplotlib.pyplot
    import matplotlib.pyplot as plt


try:
    from tabulate import tabulate 
except:
    !{sys.executable} -m pip install tabulate
    from tabulate import tabulate


try:
    from ipywidgets import Label
except:
    !{sys.executable} -m pip ipwidgets
    from ipywidgets import Label


import os


try:
    import seaborn as sns 
except:
    !{sys.executable} -m pip install seaborn
    import seaborn as sns


#from nltk.corpus import names
#from collections import Counter
#from matplotlib import pyplot as plt
#%matplotlib inline
#plt.rcParams["figure.figsize"] = [16, 6]
#plt.style.use('ggplot')




# Specify Data directory

When you run the cell below, a dialog will pop up in which you should select the "Data" directory as specified in Phase 01.

In [5]:
# Import tkinter library
try:
    import tkinter as tk
except:
    !{sys.executable} -m pip tkinter
    import tkinter as tk

# Import filedialog function
from tkinter import filedialog

# Define function to select folder and return filepath
def select_folder_from_dialog():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    selected_Folder = filedialog.askdirectory()  # Open the folder dialog
    return selected_Folder

# Define "dataDir" variable as selected folder
dataDir = select_folder_from_dialog()

# Confirm filepath
print(f"Data directory specified: {dataDir}")

Data directory specified: /Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data


'/Library'

In [ ]:
# make an analysis on the basis of the subdir structure, to get list of all the available authors, and all the books that are available
# the full bookname is formally built by the publication year and the book title, searated by an underscore 
# make a class that contains and synchronizes all functionality and structure knowlegde 
# apply this class to the existing data 

# make a panel to view and select the authorname and the full book name to work with 

In [ ]:
# define and apply a panel to view and select the authorname and the full book name to work with 

In [ ]:
# ACTION:  paste the data_directory in the inputfield 

In [ ]:
#class generic_analysis_datastructure:
    
# This cell defines a class and functions for managing project data

# 🗣️ The class and functions need to be rewritten to only check for the existence of the correct folder structure
# and give an error message if relevant folders are missing. 🗣️🚨

# using pathlib
# dataDir is the main directory of all projects of all authors 
#  


def file_exists(full_filename_string):
  filename_path = Path(full_filename_string)

  if filename_path.exists():        
    if filename_path.is_file():
      return True
    else:
      print(f"{filename_path} exists, but it is not a file.")
      return False    
  else:
    print(f"{filename_path} does not exist.")
    return False



In [ ]:


# define project dir, short filename, make project data

class ProjectData2:
  def __init__(self, data_dir, author_name, pub_book_name):
    #the_filename is full file path in the current OS
    #dataFDir is string of root dir path
    self.project_name = pub_book_name 
    self.data_dir = Path(data_dir)
    self.author_name = author_name
    self.project_dir= Path(self.data_dir/self.author_name/pub_book_name)

    self.source_dir= Path(self.project_dir/'SourceText')
    #self.source_dir_path = Path.mkdir(exist_ok=True)
    self.corpus_dir=Path(self.project_dir/'TargetCorpus')
    #self.corpus_dir.mkdir(exist_ok=True)
    self.results_dir=Path(self.project_dir/'Results')
    #self.results_dir.mkdir(exist_ok=True)
    #self.scan_project_data()
    self.all_project_dirs_exist()
    
    
  def make_project_dirs(self):
    if not self.source_dir.exists():
      self.source_dir.mkdir(exist_ok=True)
    if not self.corpus_dir.exists():
      self.corpus_dir.mkdir(exist_ok=True)
    if not self.results_dir.exists():
      self.results_dir.mkdir(exist_ok=True)
    return     


  def all_project_dirs_exist(self):
    #preetting the value of the return variable exist to False  
    data_dir_exists= self.data_dir.exists()
    if not data_dir_exists: 
      print( f"The data directory {self.dataDir}  does not exist")
    else:
      data_dir_exists = True
      results_dir_exists = self.results_dir.exists()
      if not results_dir_exists:
        print( f"The results directory {self.results_dir}  does not exist")
      else:
        results_dir_exists = True 
      corpus_dir_exists = self.corpus_dir.exists()
      if not corpus_dir_exists:
        print( f"The corpus directory {self.corpus_dir}  does not exist")
      else:
        corpus_dir_exists = True  
      
      source_dir_exists = self.source_dir.exists()
      if not source_dir_exists:
        print( f"The source directory {self.source_dir}  does not exist")
      else:  
        source_dir_exists = True      
    all_dirs_exist = data_dir_exists and source_dir_exists and results_dir_exists and corpus_dir_exists and source_dir_exists
    return all_dirs_exist

  def read_sourcetextA(self):
    self.sourceText_name = str(Path(self.source_dir/f"{self.project_name}_plaintext.txt"))
    with open(self.sourceText_name) as f: 
      rawText = f.read()
    self.plain_sourcetextA = Text(rawText,self.project_name)
    return self.plain_sourcetextA


# Load the corpus you want to find results in
  def read_corpusA(self):
    self.corpusFile_path = Path(self.corpus_dir/f"{self.project_name}_fulltext.jsonl")
    with open(self.corpusFile_path) as f:
      rawProcessedData = f.readlines()
    self.data_fulltext_jsonl = [json.loads(line) for line in rawProcessedData]
    return self.data_fulltext_jsonl



In [ ]:
import os
#def scan_book_projects(data_dir, author_name):
#    author_dir = Path(data_dir / author_name)
#    book_projects_list = [folder.name for folder in os.scandir(str(author_dir)) if folder.is_dir()]
#    return book_projects_list

def make_project_name(pub_year,book_title):
    project_name= f"{pub_year}_{book_title}" 
    return project_name

def make_pub_year(project_name):
    pub_year = project_name.split("_")[0]
    return pub_year

def scan_book_projects(data_dir, author_name):
    author_dir = os.path.join(str(data_dir), author_name)
    book_projects_list = [folder.name for folder in os.scandir(author_dir) if folder.is_dir()]
    return book_projects_list

def make_book_title(project_name):
    book_title = project_name.split("_")[1]
    return book_title    

In [ ]:
author_name='Woolf'

print(scan_book_projects(path_data_dir, author_name))



#pub_booktitle_name=

#project=ProjectData2(data_dir, author_name, pub_booktitle_name)) 

In [ ]:

class ProjectsData:
    def scan_Subdirs(self, data_Dir):
        #dataDir is a pathlib Path object
        authors_list = [folder.name for folder in os.scandir(str(data_Dir)) if folder.is_dir()]
        self.authors_list = authors_list
        return authors_list

    def __init__(self, data_Dir):
        self.data_Dir = data_Dir
        self.authors_list = self.scan_Subdirs(self.data_Dir)
        

In [ ]:
all_projects = ProjectsData(path_data_dir2)


print(all_projects.authors_list)

In [ ]:



# ACTION: 
#🚨  


instruction_line = widgets.Label("Chose your project, and press Confirm button:")

# Create a dropdown widget
authors_dropdown = widgets.Dropdown(
    options=all_projects.authors_list,
    description='Authors:'
    )

author_name= authors_dropdown.value

books_dropdown = widgets.Dropdown(
    options=scan_book_projects(path_data_dir, author_name),
    description='Books:'
    )


# Create a VBox layout with the path_input widget
# panel_layout = widgets.VBox([authors_dropdown, books_dropdown  ])

# Create a button widget for the commit action
commit_button = widgets.Button(description="Confirm")
text_label=widgets.Label(value="")
commit_box= widgets.HBox([commit_button, text_label])
panel_layout=widgets.VBox()
panel_layout.children = (instruction_line,authors_dropdown, books_dropdown, commit_box)

def author_name_changed(change):
    global author_name, books_dropdown
    
    author_name = change['new']
    books_dropdown.options = scan_book_projects(path_data_dir, author_name)
    books_dropdown.value = books_dropdown.options[0]  # Select the first book by default
    commit_button.description='Confirm'

# Attach the event handler to the value change event of authors_dropdown
authors_dropdown.observe(author_name_changed, names='value')


def input_author_field_changed(change):
    new_path = change['new']
    new_path = new_path.replace("\\", "\\\\")
    new_path = new_path.replace("'", "")
    new_path = new_path.replace('"', '')
    if Path(new_path).exists():
        instruction_data_dir_line.value = "Paste the path below:"
        text_data_dir_label.value = 'This path exists'
        commit_data_dir_button.layout.visibility = 'visible'
        commit_data_dir_button.description = 'Confirm'
    else:
        commit_data_dir_button.layout.visibility = 'hidden'
        instruction_data_dir_line.value = "Please try again. Paste the path below"
        text_data_dir_label.value = 'This path does not exist'
    
    if Path(new_path).exists():
        instruction_data_dir_line.value = "Paste the path below:"
        text_data_dir_label.value = 'This path exists'
        commit_data_dir_button.layout.visibility = 'visible'

        commit_data_dir_button.description = 'Confirm'
    else:
        commit_data_dir_button.layout.visibility = 'hidden'
        # commit_button.visible = False
        instruction_data_dir_line.value = "Please try again. Paste the path below"
        text_data_dir_label.value = 'This path does not exit' 
    # Perform actions based on the new value
    
# Attach the event handler to the value change event of dropbox.value
authors_dropdown.observe(input_author_field_changed, names='value')

def commit_button_clicked(button):
    global author_name, pub_title_name,book_proj
    instruction_line.value="Paste the path below:"
    author_name= authors_dropdown.value
    pub_title_name= books_dropdown.value
    book_proj=ProjectData2(path_data_dir, author_name, pub_title_name)   

    print( book_proj.data_dir)
    commit_button.description='Confirmed'

    #text_label.value='This path exists'
    
# Attach the event handler to the commit button
commit_button.on_click(commit_button_clicked)
# Display the panel
display(panel_layout)


In [ ]:
# This cell defines a class and functions for managing project data

# 🗣️ The class and functions need to be rewritten to only check for the existence of the correct folder structure
# and give an error message if relevant folders are missing. 🗣️🚨

# using pathlib
# dataDir is the main directory of all projects of all authors 
#  


def file_exists(full_filename_string):
  filename_path = Path(full_filename_string)

  if filename_path.exists():        
    if filename_path.is_file():
      return True
    else:
      print(f"{filename_path} exists, but it is not a file.")
      return False    
  else:
    print(f"{filename_path} does not exist.")
    return False



# define project dir, short filename, make project data

class ProjectData:
  def __init__(self,the_filename, the_dataDir):
    #the_filename is full file path in the current OS
    #dataFDir is string of root dir path
    self.filename = the_filename
    self.dataDir = Path(the_dataDir)
    #self.dataDir.mkdir(exist_ok=True)
    self.project_name = self.filename.split('.')[0] 
   
    self.make_project_data()
    self.project_dir = self.dataDir/self.name_author/self.project_name   
    #self.project_dir.mkdir(exist_ok=True) 
    
    # these should theoretically already be created in other NB 
    # But  working on an other laptop with other OS it can be handy 
    # to have these directories  written in the current OS fashion  
    #self.proj_dir= Path(self.dataDir/self.name_author/self.project_name)
    
    self.source_dir= Path(self.project_dir/'SourceText')
    #self.source_dir_path = Path.mkdir(exist_ok=True)


    self.corpus_dir=Path(self.project_dir/'TargetCorpus')
    #self.corpus_dir.mkdir(exist_ok=True)
    self.results_dir=Path(self.project_dir/'Results')
    #self.results_dir.mkdir(exist_ok=True)
    self.path=self.results_dir/self.filename 


    # Create a Path object
    # directory_path= Path(dir_name_string)

    # Check if the directory exists
    # if not( directory_path.exists() and directory_path.is_dir()):
    #   print(f"{dir_name_string} does not exist as a directory.")

    
  def make_project_data(self):
    #Split the_filename into parts based on the underscore character
    parts = self.filename.split('_')
    # The first part is the author's name
    self.name_author = parts[0]
    # The second part is the publication year
    self.publication_year = parts[1]
    # The third part is the book's name
    self.text_title = parts[2]   
    self.suffix1 = parts[3] 
    # The fourth part contains the suffix and the extension
    suffix_and_extension = parts[4]

    # Split the fourth part into the suffix and the extension based on the period character
    suffix_parts = suffix_and_extension.split('.')    
    # The first part is the hyperparSuffix
    self.hyperparSuffix = suffix_parts[0]
    # The second part is the extension
    self.extension = suffix_parts[1]
    self.project_name=f"{self.publication_year}_{self.text_title}"

    #this method checks if all of the project directorie exist. 
    # It returns true if so, and False if not

  def all_project_dirs_exist(self):
    #preetting the value of the return variable exist to False  
    data_dir_exists= self.dataDir.exists()
    if not data_dir_exists: 
      print( f"The data directory {self.dataDir}  does not exist")
    else:
      data_dir_exists = True
      results_dir_exists = self.results_dir.exists()
      if not results_dir_exists:
        print( f"The results directory {self.results_dir}  does not exist")
      else:
        results_dir_exists = True 
      corpus_dir_exists = self.corpus_dir.exists()
      if not corpus_dir_exists:
        print( f"The corpus directory {self.corpus_dir}  does not exist")
      else:
        corpus_dir_exists = True  
      
      source_dir_exists = self.source_dir.exists()
      if not source_dir_exists:
        print( f"The source directory {self.source_dir}  does not exist")
      else:  
        source_dir_exists = True 
     
    all_dirs_exist = data_dir_exists and source_dir_exists and results_dir_exists and corpus_dir_exists and source_dir_exists

    return all_dirs_exist


In [ ]:
all_projects = ProjectsData(path_data_dir)
print(all_projects.authors_list)

In [ ]:
# ACTION: copy path to corpus data JSONL file here (filename should end "_fulltext.jsonl")

# startData = "/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data/Dickens/1853_BleakHouse/TargetCorpus/Dickens_1853_BleakHouse_fulltext.jsonl"

In [ ]:
# Specify hyperparameters for matcher algorithm (adjust if desired)

thresh = 2
cut = 3
ngram = 2
mindist = 3
nostops = True

hyperparSuffix = f"_t{thresh}-c{cut}-n{ngram}-m{mindist}-{'nostops' if nostops else 'stops'}"
print(f"Suffix to be appended to results: {hyperparSuffix}")

In [ ]:
# Load the text you want to find quotations from.



sourceText = f"{sourceDir}/{projectName}_plaintext.txt"
with open(sourceText) as f: 
    rawText = f.read()

tx = Text(rawText, projectName)

# Load the corpus you want to find results in

corpusFile = f"{corpusDir}/{self.projectName}_fulltext.jsonl"
  
with open(corpusFile) as f:
    rawProcessedData = f.readlines()
data = [json.loads(line) for line in rawProcessedData]

In [ ]:
matchesTally = 0

for i, article in enumerate(data): 
    clear_output()
    print('\r', '%s matches made so far. Now matching article %s of %s' % (matchesTally, i, len(data)), end='')
    if 'numMatches' not in article: 
        articleText = Text(article['fullText'], article['id'], removeStopwords=nostops)
        article['numMatches'], article['Locations in A'], article['Locations in B'] = \
        Matcher(tx, articleText, \
                threshold=thresh, cutoff=cut, ngramSize=ngram, \
                removeStopwords=nostops, minDistance=mindist).match()
        matchesTally = matchesTally + article['numMatches']
        article['fullText'] = ''

In [ ]:
# Convert results to pandas dataframe

df = pd.DataFrame(data)

# Drop corpus full text from dataset

df = df.drop(['fullText'], axis=1)

# Extract year from date published

df2 = df["datePublished"].str.split(pat="-", n=1, expand=True).rename({0: "Year"}, axis="columns")
df = pd.concat([df, df2["Year"]], axis=1)
df = df.astype({'Year': 'int64'})

# Derive decade from year

df['Decade'] = df['Year'] - (df['Year'] % 10)

In [ ]:
# Rearrange columns to be more user-friendly

colToMove = df.pop("Year")
df.insert(2, "Year", colToMove)

colToMove = df.pop("Decade")
df.insert(3, "Decade", colToMove)

In [ ]:
# Save as JSONL file for analysis and visualization

df.to_json(path_or_buf=f"{resultsDir}/{projectName}_results{hyperparSuffix}.jsonl", orient='records', lines=True)

# Quality control

In [ ]:
# Optional: reload results file (if resuming later)

#resultsData = f"{resultsDir}/Woolf_1925_Dalloway_results_t3-c3-n2-m5-nostops.jsonl"
#df = pd.read_json(resultsData, lines=True)

In [ ]:
# Check how many items include at least 1 match

len(df.loc[df["numMatches"] >=1])

In [ ]:
# Check how many matches in total

pd.DataFrame.sum(df["numMatches"])

In [ ]:
# Check that matcher ran on all lines in the dataset - result should be 0

df.loc[pd.isnull(df['Locations in A'])]

In [ ]:
# Check some random items without matches to check for false negatives - easiest to refer to PDF

df.loc[df["numMatches"] == 0]